In [12]:
using DataFrames, LinearAlgebra, Distributions, Revise, Serialization, DebuggingUtilities

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [15]:
# Get data
df = deserialize("jls/df.jls");
dropmissing!(df);

---
#### Make economy and parameters for BLP

In [16]:
firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix(df[!, [:x1, :x2, :z1, :z2]])
D = Matrix(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 200;

In [17]:
σ = ones(size(D)[2]);
pars = m.set_Pars(K = 2, nI = nI, δs=ones(nJ,1));
serialize("jls/pars.jls", pars)

In [18]:
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );

serialize("jls/ec.jls", ec)
# serialize("/export/storage_adgandhi/MiscLi/factract/analysis/ec.jls", ec)

Economy with 417 firms and 4138 tracts.


---

### Make economy for NLLS

In [19]:
firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix(df[!, [:x1, :x2]])
D = Matrix(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 1;

ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );

serialize("jls/ec_nlls.jls", ec)

Economy with 417 firms and 4138 tracts.


---

## Computation

In [8]:
Threads.nthreads()

32

In [20]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
deltas, q_iter = m.compute_deltas(ec, pars, σ,max_iter=1000, verbose=true);
# serialize("jls/deltas.jls", deltas);

dist = NaN, iterations = 593


In [21]:
for t in ec.tracts[1:100]
    @showln sum(t.shares)
end

sum(t.shares) = 0.12069666240899499
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.17652217173427082
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.315498274559035
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.27895347196774467
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.28506394696532134
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.1807881159052969
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.17576982260358553
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.17585142894077663
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.1755174340783869
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.18643770299567702
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum(t.shares) = 0.17529189712442544
(in /mnt/staff/zhli/blp-quantities/mean_utils.ipynb:2)
sum

In [13]:
using BenchmarkTools

In [73]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.274327 seconds (519.15 k allocations: 61.624 MiB, 29.17% compilation time)


In [74]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 10
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.076545 seconds (900.14 k allocations: 101.865 MiB)


In [75]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1000
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  1.515471 seconds (22.36 M allocations: 1.873 GiB, 5.73% gc time)


In [17]:
Threads.nthreads()

16

In [18]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`